### Training workflow for DLScore version 3 <br>
Changes: <br>
<ul>
    <li>With sensoring added. (But I don't think that helps, just to give it a try!) </li>
</ul>

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import backend as K
from keras import regularizers
from keras import initializers
from keras.callbacks import EarlyStopping
from keras.utils.training_utils import multi_gpu_model
from keras.utils import plot_model
from scipy.stats import pearsonr
from sklearn.model_selection import KFold
import random
import os.path
import itertools
import pickle
import json
from tqdm import *
import glob
import re
import csv
import multiprocessing as mp
from tqdm import *

random.seed(12345)

/home/mhassan/.conda/envs/tensorflow-py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
# Sensoring outliers
def sensoring(true, pred):
    """ Sensor the predicted data to get rid of outliers"""
    mn = np.min(true)
    mx = np.max(true)
    pred = np.minimum(pred, mx)
    pred = np.maximum(pred, mn)
    
    return pred

In [3]:
def split_data(x, y, pdb_ids, valid_size=0.1, test_size=0.1):
    """Converts the pandas dataframe into a matrix.
    Splits the data into train, test and validations set.
    Returns numpy arrays"""
    # Load the indices of the non-zero columns.
    # The same indices need to be used during the evaluation of test data
    #with open("nonzero_column_indices.pickle", "rb") as f:
    #    non_zero_columns = pickle.load(f)
    # Filter the zero columns out
    #data = data[:, non_zero_columns]
    
    pdb_ids = np.array(pdb_ids)
    
    # Validation set
    val_count = int(x.shape[0]*valid_size) # Number of examples to take
    val_ids = np.random.choice(x.shape[0], val_count) # Select rows randomly
    val_x = x[val_ids, :]
    val_y = y[val_ids]
    
    # Save the pdb ids of the validation set in disk
    with open('val_pdb_ids.pickle', 'wb') as f:
        pickle.dump(pdb_ids[val_ids], f)
    
    # Remove validation set from data
    mask = np.ones(x.shape[0], dtype=bool)
    mask[val_ids] = False
    x = x[mask, :]
    y = y[mask]
    pdb_ids = pdb_ids[mask]
    
    # Test set
    test_count = int(x.shape[0]*test_size)
    test_ids = np.random.choice(x.shape[0], test_count)
    test_x = x[test_ids, :]
    test_y = y[test_ids]
    
    # Save the pdb ids of the test set in disk
    with open('test_pdb_ids.pickle', 'wb') as f:
        pickle.dump(pdb_ids[test_ids], f)
    
    # Remove test set from data
    mask = np.ones(x.shape[0], dtype=bool)
    mask[test_ids] = False
    x = x[mask, :]
    y = y[mask]
        
    return x, y, val_x, val_y, test_x, test_y

In [4]:
def train_test_split(x, y, pdb_ids, test_size=0.1):
    """Converts the pandas dataframe into a matrix.
    Splits the data into train, test and validations set.
    Returns numpy arrays"""
    # Load the indices of the non-zero columns.
    # The same indices need to be used during the evaluation of test data
    #with open("nonzero_column_indices.pickle", "rb") as f:
    #    non_zero_columns = pickle.load(f)
    # Filter the zero columns out
    #data = data[:, non_zero_columns]
    
    pdb_ids = np.array(pdb_ids)
    
    # Test set
    test_count = int(x.shape[0]*test_size)
    test_ids = np.random.choice(x.shape[0], test_count)
    test_x = x[test_ids, :]
    test_y = y[test_ids]
    
    # Save the pdb ids of the test set in disk
    with open('test_pdb_ids.pickle', 'wb') as f:
        pickle.dump(pdb_ids[test_ids], f)
    
    # Remove test set from data
    mask = np.ones(x.shape[0], dtype=bool)
    mask[test_ids] = False
    x = x[mask, :]
    y = y[mask]
        
    return x, y, test_x, test_y

In [5]:
# Build the model
def get_model(x_size, hidden_layers, dr_rate=0.5, l2_lr=0.01):
    model = Sequential()
    model.add(Dense(hidden_layers[0], activation="relu", kernel_initializer='normal', input_shape=(x_size,)))
    model.add(Dropout(0.2))
    
    for i in range(1, len(hidden_layers)):
        model.add(Dense(hidden_layers[i],
                        activation="relu",
                        kernel_initializer='normal',
                        kernel_regularizer=regularizers.l2(l2_lr),
                        bias_regularizer=regularizers.l2(l2_lr)))
        model.add(Dropout(dr_rate))
   
    model.add(Dense(1, activation="linear"))
    return(model)  

In [6]:
# def get_hidden_layers():
#     x = [128, 256, 512, 768, 1024, 2048]
#     hl = []
    
#     for i in range(1, len(x)):
#         hl.extend([p for p in itertools.product(x, repeat=i+1)])
    
#     return hl

In [35]:
def run(output_dir, serial=0):
    if serial:
        print('Running in parallel')
    else:
        print('Running standalone')
    # Create the output directory
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)

    # Preprocess the data
    pdb_ids = []
    x = []
    y = []
    with open('Data_new.csv', 'r') as f:
        reader = csv.reader(f)
        next(reader, None) # Skip the header
        for row in reader:
            pdb_ids.append(str(row[0]))
            x.append([float(i) for i in row[1:349]])
            y.append(float(row[349]))    
    x = np.array(x, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    
    # Normalize the data
    mean = np.mean(x, axis=0)
    std = np.std(x, axis=0) + 0.00001
    x_n = (x - mean) / std
    
    # Write things down
    transform  = {}
    transform['std'] = std
    transform['mean'] = mean
    with open(output_dir + 'transform.pickle', 'wb') as f:
        pickle.dump(transform, f)
    
    # Read the 'best' hidden layers
    with open("best_hidden_layers.pickle", "rb") as f:
        hidden_layers = pickle.load(f)
    
    # Determine if running all alone or in parts (if in parts, assuming 8)
    if serial:
        chunk_size = (len(hidden_layers)//8) + 1
        hidden_layers = [hidden_layers[i*chunk_size:i*chunk_size+chunk_size] for i in range(8)][serial-1]

    # Network parameters
    epochs = 100
    batch_size = 128
    keras_callbacks = [EarlyStopping(monitor='val_mean_squared_error',
                                         min_delta = 0,
                                         patience=20,
                                         verbose=0)
                       ]  

    # Split the data into training and test set
    train_x, train_y, test_x, test_y = train_test_split(x_n, y, pdb_ids, test_size=0.1)
    #train_x, train_y, val_x, val_y, test_x, test_y = split_data(x_n, y, pdb_ids)
    
    pbar = tqdm_notebook(total=len(hidden_layers),
                        desc='GPU: ' + str(serial))
    for i in range(len(hidden_layers)):
        if serial:
            model_name = 'model_' + str(serial) + '_' + str(i)
        else:
            model_name = 'model_' + str(i)
            
        # Set dynamic memory allocation in a specific gpu
        config = K.tf.ConfigProto()
        config.gpu_options.allow_growth = True
        if serial:
            config.gpu_options.visible_device_list = str(serial-1)
        K.set_session(K.tf.Session(config=config))
        
        # Build the model  
        model = get_model(train_x.shape[1], hidden_layers=hidden_layers[i])
        
        # Save the model
        with open(output_dir + model_name + ".json", "w") as json_file:
            json_file.write(model.to_json())
        if not serial:
            # If  not running with other instances then use 4 GPUs
            model = multi_gpu_model(model, gpus=4)
            
        model.compile(
            loss='mean_squared_error',
            optimizer=keras.optimizers.Adam(lr=0.001),
            metrics=[metrics.mse])
        #Save the initial weights
        ini_weights = model.get_weights()
        # 10 fold cross validation
        kf = KFold(n_splits=10)
        val_fold_score = 0.0
        train_fold_score = 0.0
        for _i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
            # Reset the weights
            model.set_weights(ini_weights)
            # Train the model
            train_info = model.fit(train_x[train_index], train_y[train_index],
                                   batch_size=batch_size,
                                   epochs=epochs,
                                   shuffle=True,
                                   verbose=0,
                                   validation_split=0.1,
                                   #validation_data=(train_x[valid_index], train_y[valid_index]),
                                   callbacks=keras_callbacks)
            
            current_val_predict = sensoring(train_y[valid_index], model.predict(train_x[valid_index])).flatten()
            current_val_r2 = pearsonr(current_val_predict, train_y[valid_index])[0]
            # If the current validation score is better then save it
            if current_val_r2 > val_fold_score:
                val_fold_score = current_val_r2
                # Save the predicted values for both the training set
                train_predict = sensoring(train_y[train_index], model.predict(train_x[train_index])).flatten()
                train_fold_score = pearsonr(train_predict, train_y[train_index])[0]
                # Save the training history
                with open(output_dir + 'history_' + model_name + '_' + str(_i) + '.pickle', 'wb') as f:
                    pickle.dump(train_info.history, f)
        
        # Save the results
        dict_r = {} 
        dict_r['hidden_layers'] = hidden_layers[i]
        dict_r['pearsonr_train'] = train_fold_score
        dict_r['pearsonr_valid'] = val_fold_score
        pred = sensoring(test_y, model.predict(test_x)).flatten()
        dict_r['pearsonr_test'] = pearsonr(pred, test_y)[0]
        #pred = sensoring(test_x, test_y, model.predict(test_x)).flatten()
        # Write the result in a file
        with open(output_dir + 'result_' + model_name + '.pickle', 'wb') as f:
            pickle.dump(dict_r, f)
        # Save the model weights
        model.save_weights(output_dir + "weights_" + model_name + ".h5")
        # Clear the session and the model from the memory
        del model
        K.clear_session()
        pbar.update()

In [36]:
output_dir = 'dl_networks_04/'

In [9]:
jobs = [mp.Process(target=run, args=(output_dir, i)) for i in range(1, 9, 1)]

In [10]:
for j in jobs:
    j.start()

Running in parallel
Running in parallel
Running in parallel
Running in parallel
Running in parallel
Running in parallel
Running in parallel
Running in parallel


HBox(children=(IntProgress(value=0, description='GPU: 4', max=13), HTML(value='')))

HBox(children=(IntProgress(value=0, description='GPU: 3', max=13), HTML(value='')))

HBox(children=(IntProgress(value=0, description='GPU: 2', max=13), HTML(value='')))

HBox(children=(IntProgress(value=0, description='GPU: 6', max=13), HTML(value='')))

HBox(children=(IntProgress(value=0, description='GPU: 5', max=13), HTML(value='')))

HBox(children=(IntProgress(value=0, description='GPU: 1', max=13), HTML(value='')))

HBox(children=(IntProgress(value=0, description='GPU: 7', max=13), HTML(value='')))

HBox(children=(IntProgress(value=0, description='GPU: 8', max=9), HTML(value='')))

/home/mhassan/.conda/envs/tensorflow-py35/lib/python3.5/site-packages/scipy/stats/stats.py:3021: RuntimeWarning: invalid value encountered in float_scalars
  r = r_num / r_den
/home/mhassan/.conda/envs/tensorflow-py35/lib/python3.5/site-packages/scipy/stats/stats.py:3021: RuntimeWarning: invalid value encountered in float_scalars
  r = r_num / r_den


### Result Analysis

In [37]:
# Get the network number and pearson coffs. of train, test and validation set in a list (in order)

model_files = sorted(glob.glob(output_dir + 'model_*'))
weight_files = sorted(glob.glob(output_dir + 'weights_*'))
result_files = sorted(glob.glob(output_dir + 'result_*'))

models = []
r2 = []
hidden_layers = []
weights = []
# net_layers = []
for mod, res, w in zip(model_files, result_files, weight_files):
    models.append(mod)
    weights.append(w)
    with open(res, 'rb') as f:
        r = pickle.load(f)
    coeff = [r['pearsonr_train'], r['pearsonr_test'], r['pearsonr_valid']]
    r2.append(coeff)
    hidden_layers.append(r['hidden_layers'])


Sort the indices according to the validation result

In [38]:
r2_ar = np.array(r2)
sorted_indices = list((-r2_ar)[:, 2].argsort())

In [39]:
sorted_r2 = [r2[i] for i in sorted_indices]

In [40]:
sorted_r2[:5]

[[0.9278997, 0.71250445, 0.7681534],
 [0.93809354, 0.6875171, 0.76786435],
 [0.9335826, 0.69356865, 0.76711303],
 [0.9088995, 0.6989541, 0.7666596],
 [0.94130564, 0.69059765, 0.76646286]]

In [41]:
sorted_models = [models[i] for i in sorted_indices]

In [42]:
sorted_models[:5]

['dl_networks_04/model_4_1.json',
 'dl_networks_04/model_5_3.json',
 'dl_networks_04/model_2_5.json',
 'dl_networks_04/model_2_3.json',
 'dl_networks_04/model_1_6.json']

In [43]:
sorted_weights = [weights[i] for i in sorted_indices]

In [44]:
sorted_weights[:5]

['dl_networks_04/weights_model_4_1.h5',
 'dl_networks_04/weights_model_5_3.h5',
 'dl_networks_04/weights_model_2_5.h5',
 'dl_networks_04/weights_model_2_3.h5',
 'dl_networks_04/weights_model_1_6.h5']

Save the lists in the disk

In [49]:
with open(output_dir + 'sorted_models.pickle', 'wb') as f:
    pickle.dump(sorted_models, f)
with open(output_dir + 'sorted_r2.pickle', 'wb') as f:
    pickle.dump(sorted_r2, f)
with open(output_dir + 'sorted_weights.pickle', 'wb') as f:
    pickle.dump(sorted_weights, f)